In [1]:
!pip install groq pandas openpyxl PyPDF2 PyMuPDF

In [3]:
import pandas as pd
import json
import os
from groq import Groq
import PyPDF2
import fitz  # PyMuPDF for better PDF handling
from typing import Dict, List, Optional
import re
from pathlib import Path

In [ ]:
GROQ_API_KEY = "api_key_goes_here"
PDF_FILE_PATH = "FPS-mock-data.pdf"
OUTPUT_EXCEL_PATH = "extracted_checkbox_data.xlsx" 

In [5]:
def extract_text_from_pdf(self, pdf_path: str) -> str:
        """
        Extract text from PDF file using multiple methods for better accuracy.
        
        Args:
            pdf_path (str): Path to PDF file
            
        Returns:
            str: Extracted text content
        """
        try:
            # Method 1: Using PyMuPDF for better text extraction
            print("Attempting PDF extraction with PyMuPDF...")
            doc = fitz.open(pdf_path)
            text = ""
            for page_num in range(len(doc)):
                page = doc[page_num]
                text += f"\n--- Page {page_num + 1} ---\n"
                text += page.get_text()
            doc.close()
            
            if text.strip():
                print(f"Successfully extracted {len(text)} characters with PyMuPDF")
                return text
                
        except Exception as e:
            print(f"PyMuPDF extraction failed: {e}")
        
        try:
            # Method 2: Fallback to PyPDF2
            print("Attempting PDF extraction with PyPDF2...")
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page_num, page in enumerate(pdf_reader.pages):
                    text += f"\n--- Page {page_num + 1} ---\n"
                    text += page.extract_text()
                    
            if text.strip():
                print(f"Successfully extracted {len(text)} characters with PyPDF2")
                return text
                
        except Exception as e:
            print(f"PyPDF2 extraction failed: {e}")
        
        print("All PDF extraction methods failed")
        return ""

In [8]:
test = extract_text_from_pdf(None, PDF_FILE_PATH)

Attempting PDF extraction with PyMuPDF...
Successfully extracted 1169 characters with PyMuPDF


In [ ]:
print(test)

In [13]:
def _fix_and_parse_json(self, json_str: str) -> Dict:
    """
    Attempt to fix common JSON issues and parse.
    """
    try:
        # Common fixes
        fixed_json = json_str.replace('\n', ' ').replace('\t', ' ')
        fixed_json = re.sub(r',\s*}', '}', fixed_json)  # Remove trailing commas
        fixed_json = re.sub(r',\s*]', ']', fixed_json)  # Remove trailing commas in arrays
        
        return json.loads(fixed_json)
    except:
        return self._create_fallback_response(json_str)

def _create_fallback_response(self, response_text: str) -> Dict:
    """
    Create a fallback response structure.
    """
    return {
        "place_of_performance": {
            "selected_option": "Manual review required",
            "content": "Please check the original document for checkbox selections"
        },
        "remuneration_invoicing": {
            "selected_options": ["Manual review required"],
            "content": "Please check the original document for checkbox selections",
            "table": {
                "included": False,
                "data": []
            }
        },
        "raw_response": response_text[:500] + "..." if len(response_text) > 500 else response_text
    }

In [14]:
client = Groq(api_key=GROQ_API_KEY)
def analyze_document_with_groq(text_content: str) -> Dict:
    """
    Use Groq API to analyze the document and identify checked checkboxes.
    
    Args:
        text_content (str): Text content from PDF
        
    Returns:
        Dict: Structured data with extracted information
    """
    prompt = f"""
    Analyze the following document content and extract information based on checked checkboxes:

    DOCUMENT CONTENT:
    {text_content}

    TASK:
    Please analyze this document and extract information for checked/selected options only:

    1. For "Place of performance" section:
        - Look for checkbox symbols like "区", "✓", "☑", "[x]", "[X]" or similar markers
        - Identify which checkbox option is marked (checked/selected)
        - Extract all relevant content for the checked option only
        - If "others" is selected, extract the complete location details (Name, Street, City, Country)

    2. For "Remuneration/Invoicing" section:
        - Look for checkbox symbols that indicate selection
        - Identify which remuneration options are checked (there can be multiple)
        - For each checked option, extract the complete content
        - If option 2 ("Remuneration based on time expended and on the hourly/daily rates") OR option 3 ("Remuneration based on time expended with upper limit") is checked, also extract the complete table with all data
        - Pay special attention to the checkbox symbol "区" which appears to indicate selection

    IMPORTANT: Only extract content for options that are actually checked/marked. Empty checkboxes should be ignored.

    Return the result as a valid JSON object with this exact structure:
    {{
        "place_of_performance": {{
            "selected_option": "description of the selected option",
            "content": "complete content for the selected option including all details"
        }},
        "remuneration_invoicing": {{
            "selected_options": ["list of selected option descriptions"],
            "content": "complete content for all selected options",
            "table": {{
                "included": true,
                "data": [
                    {{"Contractual_Service": "value", "Service_Category": "value", "Skill_level": "value", "Numbers_of_units": "value", "Unit": "value", "Price_per_unit_EUR": "value", "Total_EUR": "value"}}
                ]
            }}
        }}
    }}
    """

    try:
        print("Sending request to Groq API...")
        response = client.chat.completions.create(
            model="openai/gpt-oss-120b",  # You can also use "mixtral-8x7b-32768" or "gemma-7b-it"
            messages=[
                {
                    "role": "system", 
                    "content": "You are an expert document analyzer specializing in extracting checkbox-based content from legal and business documents. Always return valid JSON responses."
                },
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=3000
        )
        
        response_text = response.choices[0].message.content.strip()
        print(f"Received response from Groq API ({len(response_text)} characters)")
        
        # Try to extract JSON from response
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        
        if json_start != -1 and json_end != -1:
            json_str = response_text[json_start:json_end]
            try:
                parsed_data = json.loads(json_str)
                print("Successfully parsed JSON response")
                return parsed_data
            except json.JSONDecodeError as e:
                print(f"JSON parsing error: {e}")
                print(f"Attempting to fix JSON...")
                return _fix_and_parse_json(json_str)
        else:
            print("No JSON found in response")
            return _create_fallback_response(response_text)
            
    except Exception as e:
        print(f"Error with Groq API: {e}")
        return _create_fallback_response("")

In [16]:
result = analyze_document_with_groq(test)

Sending request to Groq API...
Received response from Groq API (1081 characters)
Successfully parsed JSON response


In [18]:
result

{'place_of_performance': {'selected_option': 'others (please specify)',
  'content': 'Location Name: Techzone\nStreet: Kazhakootam\nCity: Trivandrum\nCountry: India'},
 'remuneration_invoicing': {'selected_options': ['Remuneration based on time expended (see table below with cost estimate) with an upper limit of EUR 40000 EUR (net) pursuant to Section 8.2.3 of the GTC.'],
  'content': 'Remuneration based on time expended (see table below with cost estimate) with an upper limit of EUR 40000 EUR (net) pursuant to Section 8.2.3 of the GTC.',
  'table': {'included': True,
   'data': [{'Contractual_Service': 'Software engineer',
     'Service_Category': 'SCO2',
     'Skill_level': 'Expert',
     'Numbers_of_units': '40',
     'Unit': 'Man day',
     'Price_per_unit_EUR': '1000',
     'Total_EUR': '40000'}]}}}

In [19]:
def create_excel_output(extracted_data: Dict, output_path: str = "extracted_data.xlsx") -> str:
        """
        Create Excel file with extracted data.
        
        Args:
            extracted_data (Dict): Extracted and structured data
            output_path (str): Output Excel file path
            
        Returns:
            str: Path to created Excel file
        """
        try:
            # Prepare data for Excel
            place_data = extracted_data.get("place_of_performance", {})
            remuner_data = extracted_data.get("remuneration_invoicing", {})
            
            # Create main data row
            main_data = {
                "Place of Performance": place_data.get("content", ""),
                "Remuneration/Invoicing": remuner_data.get("content", ""),
                "Selected Place Option": place_data.get("selected_option", ""),
                "Selected Remuneration Options": ", ".join(remuner_data.get("selected_options", [])),
                "Table Included": remuner_data.get("table", {}).get("included", False),
                "Extraction Timestamp": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            
            # Create DataFrame for main data
            main_df = pd.DataFrame([main_data])
            
            # Create Excel writer
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                # Write main extracted data
                main_df.to_excel(writer, sheet_name='Extracted_Data', index=False)
                
                # Write table data if available
                table_data = remuner_data.get("table", {}).get("data", [])
                if table_data and isinstance(table_data, list) and len(table_data) > 0:
                    table_df = pd.DataFrame(table_data)
                    table_df.to_excel(writer, sheet_name='Remuneration_Table', index=False)
                    print(f"Table data exported: {len(table_data)} rows")
                
                # Write raw data for debugging
                raw_data = {
                    "Section": ["Place of Performance", "Remuneration/Invoicing"],
                    "Raw_Data": [str(place_data), str(remuner_data)]
                }
                raw_df = pd.DataFrame(raw_data)
                raw_df.to_excel(writer, sheet_name='Raw_Data', index=False)
            
            print(f"Excel file successfully created: {output_path}")
            return output_path
            
        except Exception as e:
            print(f"Error creating Excel file: {e}")
            return ""

In [20]:
excel_path = create_excel_output(result, OUTPUT_EXCEL_PATH)
if excel_path:
    print(f"✅ Excel file created: {excel_path}")
else:
    print("❌ Failed to create Excel file")

Table data exported: 1 rows
Excel file successfully created: extracted_checkbox_data.xlsx
✅ Excel file created: extracted_checkbox_data.xlsx
